In [37]:
import numpy as np
import nltk
from nltk.corpus import reuters
import string
import re
from random import randint
import math
import argparse, sys
import time

# W = np.loadtxt('data/W_NEG_20_DIM_300_EPOCHS_50_nonum_full_win3.txt')
npz = np.load('data/W_NEG_20_DIM_300_EPOCHS_25_nonum_full_win3_epoch75.npy.npz')
W = npz['arr_0']

In [185]:
numTokens = 0
len(vocab)
# for key, value in vocab.items():
#     numTokens += value
# numTokens

34055

In [6]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

data = "afsdfsafsdf. dr. sdf sdf sf sdf ."
print('\n-----\n'.join(tokenizer.tokenize(data)))



afsdfsafsdf.
-----
dr. sdf sdf sf sdf .


In [39]:
len(W)

300

In [40]:
len(W[0])

34055

In [41]:
def cosineDistance(x, y):
    s = np.dot(x,y)
    s = s/((np.dot(x,x))**0.5)
    s = s/((np.dot(y,y))**0.5)
    return s

def cosineDistanceString(s1, s2, W, word2id):
    x = W[word2id[s1]]
    y = W[word2id[s2]]
    return cosineDistance(x, y)

In [42]:
def readVocabulary(fileName):
    f1 = open(fileName, "r")
    vocab = {}
    for line in f1:
        v = line.strip().split('\t')
        vocab[v[0]] = int(v[1])
    f1.close()
    return vocab

def readWord2id(fileName):
    f1 = open(fileName, "r")
    word2id = {}
    for line in f1:
        v = line.strip().split('\t')
        word2id[v[0]] = int(v[1])
    f1.close()
    return word2id

def readId2word(fileName):
    f1 = open(fileName, "r")
    id2word = {}
    for line in f1:
        v = line.strip().split('\t')
        id2word[int(v[0])] = v[1]
    f1.close()
    return id2word

def readAllHashnum():
    vocab = readVocabulary('data/vocab_hashnum.txt')
    word2id = readWord2id('data/word2id_hashnum.txt')
    id2word = readId2word('data/id2word_hashnum.txt')
    return vocab, word2id, id2word

def readAllNonum():
    vocab = readVocabulary('data/vocab_nonum.txt')
    word2id = readWord2id('data/word2id_nonum.txt')
    id2word = readId2word('data/id2word_nonum.txt')
    return vocab, word2id, id2word


def readAllPrevious():
    vocab = readVocabulary('../data_word2vec/vocab.txt')
    word2id = readWord2id('../data_word2vec/word2id.txt')
    id2word = readId2word('../data_word2vec/id2word.txt')
    return vocab, word2id, id2word

In [44]:
vocab, word2id, id2word = readAllNonum()
# vocab, word2id, id2word = readAllPrevious()


In [45]:
len(word2id)

34055

In [46]:
W = W.T

In [12]:

embedding_king = W[word2id['queen']]

In [21]:
word_to_eval = "happy"
closest_word = ''
closest_word_dist = -9999
for i in range(len(W)):
    word = id2word[i]
    dist = cosineDistance(W[word2id[word_to_eval]], W[i])
    if(dist > closest_word_dist and i != word2id[word_to_eval] and i != word2id['craft']):
        closest_word_dist = dist
        closest_word = word
closest_word



'sure'

In [125]:
cosineDistanceString('good', 'planted', C, word2id)

0.14825886183046774

In [48]:
cosineDistance(W[word2id['but']],W[word2id['###']])

KeyError: '###'

In [78]:
C = np.loadtxt('data/C_NEG_15_DIM_300_EPOCHS_100_new.txt')

In [26]:
def findClosest(word_to_eval, W, word2id, id2word):
    closest_word = ''
    closest_word_dist = -9999
    for i in range(len(W)):
        word = id2word[i]
        dist = cosineDistance(W[word2id[word_to_eval]], W[i])
        if(dist > closest_word_dist and i != word2id[word_to_eval]):
            closest_word_dist = dist
            closest_word = word
    return closest_word


In [237]:
def findTopK(word_to_eval, W, word2id, id2word):
    dict = {}
    for i in range(len(W)):
        word = id2word[i]
        dist = cosineDistance(W[word2id[word_to_eval]], W[i])
        dict[word] = dist
    sorted_by_value = sorted(dict.items(), key=lambda kv: kv[1], reverse=True)
    return sorted_by_value[1:20]

In [299]:
wordToFindClosest = 'women'
ans = findTopK(wordToFindClosest, C, word2id, id2word)
print(wordToFindClosest)
for a in ans:
    print(a[0])

women
martell
wk
stabilized
lived
ceremony
rafsanjani
rio
bbls
macandrews
cadillac
refer
returning
denver
wellemeyer
13-week
ivaco
alvite
milds
barber


In [289]:
cosineDistanceString('white', 'killed', C, word2id)

0.19075806151222577

In [51]:
len(scores)

675

In [61]:
from scipy.stats import spearmanr

def evalSimlex(W, vocab):
    f = open("SimLex-999.txt", "r")
    scores = []
    i = 0
    for line in f:
        i+= 1
        if(i == 1):
            continue
        line = line.strip()
        tokens = line.split('\t')
        sample = {}
        sample['w1'] = tokens[0]
        sample['w2'] = tokens[1]
        sample['score'] = float(tokens[3])
        if sample['w1'] in vocab and sample['w2'] in vocab:
            scores.append(sample)
    predicted_scores = []
    simlex_score = []
    for sample in scores:
        w1 = sample['w1']
        w2 = sample['w2']
        predicted_scores.append(cosineDistanceString(w1, w2, W, word2id))
        simlex_score.append(sample['score'])
    corr, p_value = spearmanr(simlex_score, predicted_scores)
    return corr

def evalWordsim(W, vocab):
    f = open("data/wordsim.txt", "r")
    scores = []
    i = 0
    for line in f:
        i+= 1
        if(i == 1):
            continue
        line = line.strip()
        tokens = line.split('\t')
        sample = {}
        sample['w1'] = tokens[0]
        sample['w2'] = tokens[1]
        sample['score'] = float(tokens[2])
        if sample['w1'] in vocab and sample['w2'] in vocab:
            scores.append(sample)
    predicted_scores = []
    wordsim_score = []
    for sample in scores:
        w1 = sample['w1']
        w2 = sample['w2']
        predicted_scores.append(cosineDistanceString(w1, w2, W, word2id))
        wordsim_score.append(sample['score'])
    corr, p_value = spearmanr(wordsim_score, predicted_scores)
    return corr

In [253]:
npz = np.load('data/W_NEG_20_DIM_300_EPOCHS_25_nonum_full_win3_epoch75.npy.npz')
W = npz['arr_0']
npz = np.load('data/C_NEG_20_DIM_300_EPOCHS_25_nonum_full_win3_epoch75.npy.npz')
C = npz['arr_0']

# W = np.loadtxt('data/W_NEG_15_DIM_300_EPOCHS_500_t6.txt')
# C = np.loadtxt('data/C_NEG_15_DIM_300_EPOCHS_500_t6.txt')
W=W.T
WC = np.concatenate((W, C), axis = 1)
vocab, word2id, id2word = readAllNonum()

In [254]:
len(W)

34055

In [255]:
print('W')
print('Simlex', evalSimlex(W, vocab))
print('Wordsim', evalWordsim(W, vocab))

print('C')
print('Simlex', evalSimlex(C, vocab))
print('Wordsim', evalWordsim(C, vocab))

print('Avg')
print('Simlex', evalSimlex((W+C)/2, vocab))
print('Wordsim', evalWordsim((W+C)/2, vocab))

print('Concatenated')
print('Simlex', evalSimlex(WC, vocab))
print('Wordsim', evalWordsim(WC, vocab))

W
Simlex 0.121804617502587
Wordsim 0.20161243598711406
C
Simlex 0.13953518501066306
Wordsim 0.30418429005041586
Avg
Simlex 0.10508361488406495
Wordsim 0.2772922755589194
Concatenated
Simlex 0.15152822358615842
Wordsim 0.2872126820051131


In [84]:
len(WC[0])

600

In [82]:
cosineDistance(C[1], W[1])

-0.011782604157235591

In [179]:
vocab['the']

69245

In [182]:
numGood = 0
for key, value in vocab.items():
    if(value >= 2):
        numGood+= 1
numGood

20048